## Importing Libraries

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window

**1596. The Most Frequently Ordered Products for Each Customer (Medium)**

**Table: Customers**

| Column Name   | Type    |
|---------------|---------|
| customer_id   | int     |
| name          | varchar |

customer_id is the column with unique values for this table.
This table contains information about the customers.
 
**Table: Orders**

| Column Name   | Type    |
|---------------|---------|
| order_id      | int     |
| order_date    | date    |
| customer_id   | int     |
| product_id    | int     |

order_id is the column with unique values for this table.
This table contains information about the orders made by customer_id.
No customer will order the same product more than once in a single day.
 
**Table: Products**

| Column Name   | Type    |
|---------------|---------|
| product_id    | int     |
| product_name  | varchar |
| price         | int     |

product_id is the column with unique values for this table.
This table contains information about the products.
 
**Write a solution to find the most frequently ordered product(s) for each customer.**

The result table should have the product_id and product_name for each customer_id who ordered at least one order.

Return the result table in any order.

The result format is in the following example.

Example 1:

Input: 

**Customers table:**
| customer_id | name  |
|-------------|-------|
| 1           | Alice |
| 2           | Bob   |
| 3           | Tom   |
| 4           | Jerry |
| 5           | John  |

**Orders table:**
| order_id | order_date | customer_id | product_id |
|----------|------------|-------------|------------|
| 1        | 2020-07-31 | 1           | 1          |
| 2        | 2020-07-30 | 2           | 2          |
| 3        | 2020-08-29 | 3           | 3          |
| 4        | 2020-07-29 | 4           | 1          |
| 5        | 2020-06-10 | 1           | 2          |
| 6        | 2020-08-01 | 2           | 1          |
| 7        | 2020-08-01 | 3           | 3          |
| 8        | 2020-08-03 | 1           | 2          |
| 9        | 2020-08-07 | 2           | 3          |
| 10       | 2020-07-15 | 1           | 2          |

**Products table:**
| product_id | product_name | price |
|------------|--------------|-------|
| 1          | keyboard     | 120   |
| 2          | mouse        | 80    |
| 3          | screen       | 600   |
| 4          | hard disk    | 450   |

**Output:** 
| customer_id | product_id | product_name |
|-------------|------------|--------------|
| 1           | 2          | mouse        |
| 2           | 1          | keyboard     |
| 2           | 2          | mouse        |
| 2           | 3          | screen       |
| 3           | 3          | screen       |
| 4           | 1          | keyboard     |

**Explanation:** 
- Alice (customer 1) ordered the mouse three times and the keyboard one time, so the mouse is the most frequently ordered product for them.
- Bob (customer 2) ordered the keyboard, the mouse, and the screen one time, so those are the most frequently ordered products for them.
- Tom (customer 3) only ordered the screen (two times), so that is the most frequently ordered product for them.
- Jerry (customer 4) only ordered the keyboard (one time), so that is the most frequently ordered product for them.
- John (customer 5) did not order anything, so we do not include them in the result table.

In [0]:
customers_data_1596 = [
    (1, "Alice"),
    (2, "Bob"),
    (3, "Tom"),
    (4, "Jerry"),
    (5, "John"),
]

customers_columns_1596 = ["customer_id", "name"]
customers_df_1596 = spark.createDataFrame(customers_data_1596, customers_columns_1596)
customers_df_1596.show()

orders_data_1596 = [
    (1, "2020-07-31", 1, 1),
    (2, "2020-07-30", 2, 2),
    (3, "2020-08-29", 3, 3),
    (4, "2020-07-29", 4, 1),
    (5, "2020-06-10", 1, 2),
    (6, "2020-08-01", 2, 1),
    (7, "2020-08-01", 3, 3),
    (8, "2020-08-03", 1, 2),
    (9, "2020-08-07", 2, 3),
    (10, "2020-07-15", 1, 2),
]

orders_columns_1596 = ["order_id", "order_date", "customer_id", "product_id"]
orders_df_1596 = spark.createDataFrame(orders_data_1596, orders_columns_1596)
orders_df_1596.show()

products_data_1596 = [
    (1, "keyboard", 120),
    (2, "mouse", 80),
    (3, "screen", 600),
    (4, "hard disk", 450),
]

products_columns_1596 = ["product_id", "product_name", "price"]
products_df_1596 = spark.createDataFrame(products_data_1596, products_columns_1596)
products_df_1596.show()

+-----------+-----+
|customer_id| name|
+-----------+-----+
|          1|Alice|
|          2|  Bob|
|          3|  Tom|
|          4|Jerry|
|          5| John|
+-----------+-----+

+--------+----------+-----------+----------+
|order_id|order_date|customer_id|product_id|
+--------+----------+-----------+----------+
|       1|2020-07-31|          1|         1|
|       2|2020-07-30|          2|         2|
|       3|2020-08-29|          3|         3|
|       4|2020-07-29|          4|         1|
|       5|2020-06-10|          1|         2|
|       6|2020-08-01|          2|         1|
|       7|2020-08-01|          3|         3|
|       8|2020-08-03|          1|         2|
|       9|2020-08-07|          2|         3|
|      10|2020-07-15|          1|         2|
+--------+----------+-----------+----------+

+----------+------------+-----+
|product_id|product_name|price|
+----------+------------+-----+
|         1|    keyboard|  120|
|         2|       mouse|   80|
|         3|      screen|  6

In [0]:
order_counts_df_1596 = orders_df_1596\
                            .groupBy("customer_id", "product_id")\
                                .agg(count("*").alias("order_count"))

In [0]:
max_counts_df_1596 = order_counts_df_1596\
                            .groupBy("customer_id")\
                                .agg(max("order_count").alias("max_count"))

In [0]:
top_products_df_1596 = order_counts_df_1596\
                                .join(max_counts_df_1596, on="customer_id", how="inner")\
                                    .filter("order_count = max_count")


In [0]:
top_products_df_1596\
    .join(products_df_1596, on="product_id", how="inner")\
    .select("customer_id", "product_id", "product_name").show()

+-----------+----------+------------+
|customer_id|product_id|product_name|
+-----------+----------+------------+
|          2|         2|       mouse|
|          3|         3|      screen|
|          4|         1|    keyboard|
|          1|         2|       mouse|
|          2|         1|    keyboard|
|          2|         3|      screen|
+-----------+----------+------------+

